# Run quantum accurate geometry optimisation on a small molecule

This notebook shows how to geometry optimise a small molecule using RIMP2 accurate gradients.

In [1]:
import os
from glob import glob
from pathlib import Path
import json

import rush

## 0) Setup

In [2]:
# Define our project information
DESCRIPTION = "rush-py qm geo opt notebook"
TAGS = ["qdx", "rush-py-v2", "demo", "qm_geo_opt"]
WORK_DIR = Path.home() / "qdx" / "qm_geo_opt"

In [3]:
# |hide
if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=True)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys
os.chdir(WORK_DIR)

## 0.1) Initialize our rush client and fetch available module paths

In [6]:
# By using the `build_provider_with_functions` method, we will also build helper functions calling each module
client = await rush.build_provider_with_functions(
    access_token=RUSH_TOKEN, url=RUSH_URL, batch_tags=TAGS
)

In [7]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

## 0.2) Get qm_geo_opt rush module

In [8]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducable
# This will be done automatically if you use the `build_provider_with_functions` method
modules = await client.get_latest_module_paths()

In [9]:
module_name = "qm_geo_opt"
module_path = modules[module_name]
print(module_path)

github:talo/qm_geo_opt/416a746889e5f67e7a1505812b64c16c0a0c1f87#qm_geo_opt_tengu


In [10]:
help(client.qm_geo_opt)

Help on function qm_geo_opt in module rush.provider:

async qm_geo_opt(*args: [<class 'pathlib.Path'>, dict[str, ~T]], target: rush.graphql_client.enums.ModuleInstanceTarget | None = <ModuleInstanceTarget.NIX_SSH_GPU: 'NIX_SSH_GPU'>, resources: rush.graphql_client.input_types.ModuleInstanceResourcesInput | None = ModuleInstanceResourcesInput(gpus=0, gpu_mem=None, gpu_mem_units=None, cpus=None, nodes=None, mem=None, mem_units=None, storage=10, storage_units=<MemUnits.MB: 'MB'>, walltime=None, storage_mounts=None), tags: list[str] | None = None, restore: bool | None = None) -> [<class 'pathlib.Path'>]
    geometry optimise molecule with quantum accuracy
    
    Module version: github:talo/qm_geo_opt/416a746889e5f67e7a1505812b64c16c0a0c1f87#qm_geo_opt_tengu
    
    QDX Type Description:
    
        input: @Conformer;
    
        parameters: {
    
        basis_set:string,
    
        conv_threshold:f64,
    
        max_iterations:u32,
    
        use_internal_coords:bool
    
    

# 1) Run Geometry Optimisation

## 1.0) Create input geometry
We will be creating a qdxf conformer object from an unoptimised cyclobutane xyz

In [11]:
def xyz_to_qdxf_topology(xyz_data : str):
  symbols = []
  geometry = []
  for line in xyz_data.splitlines()[3:]:
    symbol, x, y, z = line.split()
    symbols.append(symbol)
    geometry.extend([float(x), float(y), float(z)])
  return {
    "topology": {
      "symbols": symbols,
      "geometry": geometry
    }
  }

def qdxf_topology_to_xyz(qdxf_in : dict, name : str):
  symbols = qdxf_in["topology"]["symbols"]
  geometry = qdxf_in["topology"]["geometry"]
  xyz_data = f"{len(symbols)}\n{name}\n"
  for i in range(len(symbols)):
    symbol = symbols[i]
    x, y, z = geometry[3*i:3*i+3]
    xyz_data += f"  {symbol}"
    for coord in [x, y, z]:
      xyz_data += "     "
      if (coord >= 0):
        xyz_data += " "
      xyz_data += f"{(10 * coord):.4f}"
    xyz_data += "\n"
  return xyz_data

In [33]:
cyclobutane_xyz = '''12
cyclobutane
  C     -0.750      0.452     -0.417
  C     -0.696     -0.588      0.609
  C      0.820     -0.678      0.537
  C      0.892      0.417     -0.428
  H     -1.285      1.273      0.066
  H     -1.328      0.080     -1.263
  H     -1.225     -1.507      0.366
  H     -1.029     -0.162      1.555
  H      1.158     -1.594      0.054
  H      1.310     -0.477      1.488
  H      1.432      0.009     -1.290
  H      1.506      1.236     -0.056
'''

In [34]:
cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

In [35]:
# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
  json.dump(cyclobutane_qdxf, f)

## 1.1) Create geometry optimisation parameters 

In [41]:
params = {
  "use_internal_coords": True,
  "max_iterations": 100,
  "conv_threshold": 1e-4,
  "basis_set": "STO-3G"
}

## 1.1) Call geometry optimisation over input geometry

In [42]:
(geo_opt_out,) = await client.qm_geo_opt(cyclobutane_qdxf_input_file, params, target="NIX_SSH_GPU")
print(await geo_opt_out.get())

2024-01-30 13:54:35,088 - rush - INFO - Argument f2d15c6f-59cc-425e-8969-f7922365199b is now RESOLVING
2024-01-30 13:54:44,964 - rush - INFO - Argument f2d15c6f-59cc-425e-8969-f7922365199b is now ADMITTED
2024-01-30 13:54:57,625 - rush - INFO - Argument f2d15c6f-59cc-425e-8969-f7922365199b is now DISPATCHED


Exception: (<ModuleFailureReason.RUN: 'RUN'>, ModuleInstanceCommonFailureContext(stdout='\nWriting "/atom_labels" to: "./temp/temp_geo_opt.hdf5"\n/build/n7ynpjxm49wr9165i7nj7fdmfp3n7sk2-source/src/methods/geometry_optimisation/coordinate_systems/internal_system.cpp L194:\n\tAssertion \'is_fully_connected(topology, internal_coords)\' failed: interfragment coordinates not implemented\nSkipping MPI win free due to uncaught exception\n********************************************************************************\nA runtime error has occurred on rank 1!\nThe error message is printed below:\n/build/n7ynpjxm49wr9165i7nj7fdmfp3n7sk2-source/src/methods/geometry_optimisation/coordinate_systems/internal_system.cpp L194:\n\tAssertion \'is_fully_connected(topology, internal_coords)\' failed: interfragment coordinates not implemented\n********************************************************************************', stderr='--------------------------------------------------------------------------

## 1.2) Check calculation converged

In [24]:
(topologies_out, results_out) = await geo_opt_out.get()
assert(results_out["converged"])

Exception: (<ModuleFailureReason.RUN: 'RUN'>, ModuleInstanceCommonFailureContext(stdout='\nWriting "/atom_labels" to: "./temp/temp_geo_opt.hdf5"\n/build/n7ynpjxm49wr9165i7nj7fdmfp3n7sk2-source/src/methods/geometry_optimisation/coordinate_systems/internal_system.cpp L194:\n\tAssertion \'is_fully_connected(topology, internal_coords)\' failed: interfragment coordinates not implemented\nSkipping MPI win free due to uncaught exception\n********************************************************************************\nA runtime error has occurred on rank 1!\nThe error message is printed below:\n/build/n7ynpjxm49wr9165i7nj7fdmfp3n7sk2-source/src/methods/geometry_optimisation/coordinate_systems/internal_system.cpp L194:\n\tAssertion \'is_fully_connected(topology, internal_coords)\' failed: interfragment coordinates not implemented\n********************************************************************************', stderr='--------------------------------------------------------------------------

# 2) Visualise pre-optimised and optimised geometries

In [25]:
import py3Dmol

In [26]:
pre_opt_view = py3Dmol.view()
pre_opt_view.addModel(cyclobutane_xyz, "xyz")
pre_opt_view.setStyle({'stick':{}})
pre_opt_view.zoomTo()
pre_opt_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [27]:
(topologies_out, _) = await geo_opt_out.get()
opt_cyclobutane_xyz = qdxf_topology_to_xyz(topologies_out[-1], "optimised cyclobutane")

opt_view = py3Dmol.view()
opt_view.addModel(opt_cyclobutane_xyz, "xyz", {})
opt_view.setStyle({'stick':{}})
opt_view.zoomTo()
opt_view.show()

Exception: (<ModuleFailureReason.RUN: 'RUN'>, ModuleInstanceCommonFailureContext(stdout='\nWriting "/atom_labels" to: "./temp/temp_geo_opt.hdf5"\n/build/n7ynpjxm49wr9165i7nj7fdmfp3n7sk2-source/src/methods/geometry_optimisation/coordinate_systems/internal_system.cpp L194:\n\tAssertion \'is_fully_connected(topology, internal_coords)\' failed: interfragment coordinates not implemented\nSkipping MPI win free due to uncaught exception\n********************************************************************************\nA runtime error has occurred on rank 1!\nThe error message is printed below:\n/build/n7ynpjxm49wr9165i7nj7fdmfp3n7sk2-source/src/methods/geometry_optimisation/coordinate_systems/internal_system.cpp L194:\n\tAssertion \'is_fully_connected(topology, internal_coords)\' failed: interfragment coordinates not implemented\n********************************************************************************', stderr='--------------------------------------------------------------------------